In [1]:
import torch
import numpy as np

In [2]:
stats=np.load('allStats20210512.npy')
stats.shape

(1609, 30)

In [3]:
h=['PA','AB','R','H','2B','3B','HR',
           'RBI','BB','IBB','SO','HBP','SH','SF','GDP','SB','CS']#batting only
hBvsP=['PA2','AB2','H2', '1B2','2B2','3B2','HR2','BB2','SO2','AVG2','OBP2','SLG2'] #BvsP
hTotal=h+hBvsP
features=stats[:,len(h):-1]
labels=stats[:,-1]

In [4]:
from sklearn.model_selection import train_test_split
features=torch.tensor(features)
labels=torch.tensor(labels)

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.1, random_state=42)


In [6]:
from torch import nn
import torch.optim as optim

device = torch.device('cpu')
if torch.cuda.is_available():
    device = torch.device('cuda')

net = nn.Sequential(
      nn.Linear(features.shape[1], 5),
      nn.ReLU(),
      nn.Linear(5, 3),
      nn.ReLU(),
      nn.Linear(3, 1),
      nn.Sigmoid()
      ).to(device)

net=net.double()
epochs=100
losses=[]

correct=0
count=0
for epoch in range(epochs):
    for sample, label in zip(X_train, y_train):
#         print(sample, label)
        output=net(sample.double())

        net.zero_grad()


        target = label.double()
        target = target.view(1, -1)  # make it the same shape as output
        criterion = nn.BCELoss()

        optimizer = optim.SGD(net.parameters(), lr=0.01)

        # in your training loop:
        optimizer.zero_grad()   # zero the gradient buffers

        loss = criterion(output, target)
        loss.backward()
        optimizer.step()    # Does the update
        losses.append(loss)
        count+=1
        if (output>0.5 and label==1) or (output<0.5 and label==0):
            correct+=1
#         print('output=', output.item(), ' label = ', label.item())
    acc=correct/count
    print('epoch=' ,epoch, 'avg loss=', torch.mean(torch.tensor(losses)), 'acc = ', acc)



/home/adimn/anaconda3/envs/venv-base/lib/python3.7/site-packages/torch/nn/modules/loss.py:529: UserWarning: Using a target size (torch.Size([1, 1])) that is different to the input size (torch.Size([1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


epoch= 0 avg loss= tensor(0.6076, dtype=torch.float64) acc =  0.17196132596685082
epoch= 1 avg loss= tensor(0.6033, dtype=torch.float64) acc =  0.3011049723756906
epoch= 2 avg loss= tensor(0.5986, dtype=torch.float64) acc =  0.3710865561694291
epoch= 3 avg loss= tensor(0.5928, dtype=torch.float64) acc =  0.3786256906077348
epoch= 4 avg loss= tensor(0.5865, dtype=torch.float64) acc =  0.3791436464088398
epoch= 5 avg loss= tensor(0.5809, dtype=torch.float64) acc =  0.37419429097605894
epoch= 6 avg loss= tensor(0.5764, dtype=torch.float64) acc =  0.3724348855564325
epoch= 7 avg loss= tensor(0.5723, dtype=torch.float64) acc =  0.3723238950276243
epoch= 8 avg loss= tensor(0.5694, dtype=torch.float64) acc =  0.37093308778391654
epoch= 9 avg loss= tensor(0.5669, dtype=torch.float64) acc =  0.3695441988950276
epoch= 10 avg loss= tensor(0.5648, dtype=torch.float64) acc =  0.3685961828227022
epoch= 11 avg loss= tensor(0.5630, dtype=torch.float64) acc =  0.3713743093922652
epoch= 12 avg loss= ten

In [80]:
features.shape, labels.shape

(torch.Size([1565, 29]), torch.Size([1565]))